In [ ]:
import pandas as pd
import numpy as np
import os
import pdfplumber as pdfp
import tabula
import re
import PyPDF2
import requests
import datetime
from datetime import datetime

In [ ]:
# diretório
diretorio = r'C:\Users\Computadores Gamer\OneDrive\Área de Trabalho\contas_ape'
diretorio = diretorio.replace('\\','/')
os.chdir(diretorio)

print(os.getcwd())

In [ ]:
# arquivos
arquivos = os.listdir()
print(arquivos)

In [ ]:
# parametros para donwload do arquivos (mês e ano)
ano = '23'
mes = '8'

# selecao dos arquivos de acordo com os parâmetros
arquivos_corretos = []
for i in arquivos:
    if i[:-4].endswith(mes + ano):
        arquivos_corretos.append(i)

print(arquivos_corretos)


In [ ]:
# download dos arquivos (precisa melhorar, como vou saber que o arquivo 1 é o boleto_enel?)
boleto_condominio = pdfp.open(arquivos_corretos[0])    
boleto_enel = pdfp.open(arquivos_corretos[1])  
boleto_quintoandar = pdfp.open(arquivos_corretos[2])  


# arquivos_baixados = []
# for arquivo in arquivos_corretos:
#     with open(arquivo, 'rb') as file:
#         nome_variavel = 'conteudo_' + arquivo.split('.')[0]  # atribui o nome da variável a partir do nome do arquivo
#         conteudo = file.read()
#         globals()[nome_variavel] = conteudo  # atribui o conteúdo do arquivo à variável global com o nome adequado
#         arquivos_baixados.append(nome_variavel)

# print(arquivos_baixados)

In [ ]:
# paginas a serem lidas - condominio
boleto_condominio.pages
pagina_condominio = boleto_condominio.pages[0]
print(pagina_condominio)

In [ ]:
# extraindo texto - boleto condominio
texto_condominio = pagina_condominio.extract_text().split('\n')
texto_condominio


In [ ]:
# linhas a serem lidas
for i in texto_condominio:
    if 'CONDOMINIO' in i:
        print (i)
        indice = texto_condominio.index(i)
        print(indice)

In [ ]:
# obtencao do valor_condominio
valor_condominio = texto_condominio[indice].split()[3] # como há um indice só nao preciso movimentar esse indice
valor_condominio = valor_condominio.replace('.','')
valor_condominio = valor_condominio.replace(',','.')
valor_condominio = float(valor_condominio)

print(pagina_condominio)

In [ ]:
for i in texto_condominio:
    if 'Vencimento' in i:
        print (i)
        indice = texto_condominio.index(i)
        print(indice)

In [ ]:
vencimento_condominio = texto_condominio[indice-34].split()[2] # o ultimo indice é o 36, portanto preciso subir 34 linhas pra chegar no valor da fatura do indice 2
vencimento_condominio = datetime.strptime(vencimento_condominio, '%d/%m/%Y')
vencimento_condominio = vencimento_condominio.date() # arrancar o horário

print(vencimento_condominio)

In [44]:
# paginas - boleto enel
boleto_enel.pages
pagina_enel = boleto_enel.pages[1]
print(pagina_enel)

<Page:2>


In [47]:
# extraindo texto - boleto enel
texto_enel = pagina_enel.extract_text().split('\n')
texto_enel

['2',
 '- Segunda via de fatura',
 '25 AGO 2023 AGO 2023 12 SET 2023',
 'PREFEITURA DO MUNICÍPIO 0800 77 90 156',
 '- CLIENTE, PAGUE PREFERENCIALMENTE NOS CANAIS',
 'ELETRÔNICOS',
 '- CADASTRE SUA FATURA EM DÉBITO AUTOMÁTICO',
 'UTILIZANDO O NUMERO DA CONTA CONFORME EXIBIDO',
 'AO LADO',
 '- CONTRIBUA COM O MEIO AMBIENTE, SEJA DIGITAL E',
 'SOLICITE O RECEBIMENTO DE SUA FATURA POR E-MAIL E',
 'ECONOMIZE PAPEL. ACESSE NOSSO',
 '502916928898 25 AGO 2023 AGO 2023',
 '52170381 12 SET 2023 202,62',
 'MATEUS RODRIGUES DE SOUZA',
 '- ENCARGOS POR ATRASO SERÃO COBRADOS NA PRÓXIMA FATURA',
 '- APROVEITE OS BENEFÍCIOS DO DÉBITO AUTOMÁTICO',
 '836100000022 026200481005 073999787317 002192878284']

In [48]:
# verificando onde esta o valor
for i in texto_enel:
    if 'BENEFÍCIOS ' in i:
        print(i)
        indice_enel = texto_enel.index(i)
        print(indice_enel)
    

    

- APROVEITE OS BENEFÍCIOS DO DÉBITO AUTOMÁTICO
16


In [46]:
# chegando na linha do valor da fatura e extraindo o valor
valor_enel = texto_enel[indice_enel-3].split(' ')
valor_enel = valor_enel[4].replace(',','.')
valor_enel = float(valor_enel)

In [54]:
# data de vencimento - boleto enel
vencimento_enel = texto_enel[indice_enel-3].split(' ')


['52170381', '12', 'SET', '2023', '202,62']

In [ ]:
# paginas - quinto andar